# 빅콘테스트- 챔피언리그
---

### 2018-08-20 ~ 2018-09-14
---

In [1]:
# 모듈 임포트
import pandas as pd
import numpy as np
import tensorflow as tf

## 1. 데이터 불러오기 및 데이터 선처리
---

### 데이터 불러오기

In [2]:
# 트레이닝 데이터 로드
x_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_activity.csv').sort_values(by='acc_id', ascending=True)
y_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_label.csv').sort_values(by='acc_id', ascending=True)

In [3]:
# 데이터셋 정보
print("================ X 정보 ================")
x_data.info()
print("\n================ Y 정보 ================")
y_data.info()

================ X 정보 ================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 440323 entries, 0 to 440322
Data columns (total 38 columns):
wk                          440323 non-null int64
acc_id                      440323 non-null object
cnt_dt                      440323 non-null int64
play_time                   440323 non-null float64
npc_exp                     440323 non-null float64
npc_hongmun                 440323 non-null float64
quest_exp                   440323 non-null float64
quest_hongmun               440323 non-null float64
item_hongmun                440323 non-null float64
game_combat_time            440323 non-null float64
get_money                   440323 non-null float64
duel_cnt                    440323 non-null float64
duel_win                    440323 non-null float64
partybattle_cnt             440323 non-null float64
partybattle_win             440323 non-null float64
cnt_enter_inzone_solo       440323 non-null float64
cnt_enter_inzone_light 

In [4]:
# 트레이닝데이터 아이디별로 그룹화 및 평균
# 데이터의 시계열 특성이 사라지지만, 빨리 단순 회귀 모형을 볼 수 있음
x_data = x_data.groupby(by='acc_id').mean().values
y_data = y_data.label.values

In [5]:
#트레이닝셋, 발리데이션셋 구분
x_train = x_data[0:int(0.7 * len(x_data)),:]
y_train = y_data[0:int(0.7 * len(x_data))]

x_cv = x_data[int(0.7 * len(x_data)):,:]
y_cv = y_data[int(0.7 * len(x_data)):]

### y label encoding

In [6]:
#한국어로 된 레이블을 숫자로 인코딩
idx2label = list(set(y_train))
label2idx = {c:i for i,c in enumerate(idx2label)}

#new y_train & y_cv
y_train = [label2idx[v] for v in y_train]
y_cv = [label2idx[v] for v in y_cv]

## 2. constants
---

In [7]:
# 레이블 개수
num_feature = len(x_train[0])
learning_rate = 0.01
label = len(idx2label)

## 3. Tensorflow
---

### placeholder 정의

In [8]:
X = tf.placeholder(tf.float32, [None, num_feature])
Y = tf.placeholder(tf.int32, [None])

In [9]:
#Y_one_hot
Y_one_hot = tf.one_hot(Y, label)

In [10]:
#Parameter
w = tf.Variable(tf.random_normal([num_feature, label]), name = "weight")
b = tf.Variable(tf.random_normal([label]), name = "bias")

In [11]:
logits = tf.matmul(X, w) + b

In [12]:
#Cost Function - softmax cross entropy
with tf.name_scope("cost") as scope:
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost)
    tf.summary.scalar("cost", cost)

In [13]:
#Train - Gradient Descent
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

In [14]:
#Prediction
prediction = tf.argmax(logits, -1)

In [15]:
#Accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y_one_hot,-1)), tf.float32))
    tf.summary.scalar("accuracy", accuracy)

## 저장
---

In [16]:
saver = tf.train.Saver()

## 학습!
---

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./01_simple_linear_regression', sess.graph)
    
    for step in range(2000):
        sess.run(train, feed_dict={X:x_train, Y:y_train})
        
        if step % 20 == 0:
            summary = sess.run(merged, feed_dict={X:x_train, Y:y_train})
            writer.add_summary(summary, step)
            
    for p, y in zip(sess.run(prediction, feed_dict={X:x_cv}), y_cv):
        print("Prediction {} True_Y {} Predicted_Y {}".format(p,y,idx2label[p]))

    print(sess.run(accuracy,feed_dict={X:x_cv,Y:y_cv}))
    
    saver.save(sess, './01_simple_linear_regression/model.ckpt')
    
    writer.close()


Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 0 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 3 True_Y 2 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 3 Predi

Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 2 True_Y 1 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 0 True_Y 2 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 0 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 1 Pred

Prediction 0 True_Y 0 Predicted_Y retained
Prediction 0 True_Y 1 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 2 True_Y 3 Predicted_Y week
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 0 True_Y 3 Predicted_Y retained
Prediction 2 True_Y 1 Predicted_Y week
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 Tr

Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 3 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 0 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 0 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 2 Predict

Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 2 True_Y 0 Predicted_Y week
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 1 Predicted_Y week
Prediction 3 True_Y 0 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 

...

Prediction 0 True_Y 0 Predicted_Y retained
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 0 Predicted_Y month
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 3 Predicted_Y month
Prediction 0 True_Y 1 Predicted_Y retained
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 0 True_Y 3 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 3 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 0 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 3 Predi

Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 0 Predicted_Y week
Prediction 2 True_Y 3 Predicted_Y week
Prediction 3 True_Y 3 Predicted_Y month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 2 True_Y 0 Predicted_Y week
Prediction 3 True_Y 2 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 0 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 2 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 2 True_Y 3 Predicted_Y week
Prediction 2 True_Y 3 Predicted_Y week
Prediction 2 True_Y 3 Predicted_Y week
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 0 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 1 Predicted_Y month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 3 True_Y 2 Predicted_Y month
Pred

Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 3 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 2 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 2 Predicted_Y retained
Prediction 2 True_Y 2 

Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 2 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 0 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 3 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 3 True_Y 0 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 3 True_Y 0 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 3 Predicted_Y week
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 3 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 1 Pre

Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 2 Predicted_Y month
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 2 True_Y 3 Predicted_Y week
Prediction 3 True_Y 1 Predicted_Y month
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 1 Predicted_Y month
Prediction 0 True_Y 0 Predicted_Y retained
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 3 True_Y 2 Predicted_Y month
Prediction 1 True_Y 1 Predicted_Y 2month
Prediction 1 True_Y 3 Predicted_Y 2month
Prediction 3 True_Y 2 Predicted_Y month
Prediction 1 True_Y 2 Predicted_Y 2month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 3 True_Y 3 Predicted_Y month
Prediction 2 True_Y 2 Predicted_Y week
Prediction 2 True_Y 2 Predi

## 테스트
---

### 테스트데이터 불러오기

In [18]:
# 테스트 데이터 로드
x_test_temp = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_activity.csv').sort_values(by='acc_id', ascending=True)

In [19]:
# 데이터셋 정보
print("================ X 정보 ================")
x_test_temp.info()

================ X 정보 ================
<class 'pandas.core.frame.DataFrame'>
Int64Index: 175631 entries, 47383 to 154085
Data columns (total 38 columns):
wk                          175631 non-null int64
acc_id                      175631 non-null object
cnt_dt                      175631 non-null int64
play_time                   175631 non-null float64
npc_exp                     175631 non-null float64
npc_hongmun                 175631 non-null float64
quest_exp                   175631 non-null float64
quest_hongmun               175631 non-null float64
item_hongmun                175631 non-null float64
game_combat_time            175631 non-null float64
get_money                   175631 non-null float64
duel_cnt                    175631 non-null float64
duel_win                    175631 non-null float64
partybattle_cnt             175631 non-null float64
partybattle_win             175631 non-null float64
cnt_enter_inzone_solo       175631 non-null float64
cnt_enter_inzone_li

In [20]:
#label
label = []

In [21]:
#value
x_test = x_test_temp.groupby(by='acc_id').mean().values

In [22]:
with tf.Session() as sess:
    saver.restore(sess, './01_simple_linear_regression/model.ckpt')
    
    for v in sess.run(prediction, feed_dict={X:x_test}) :
        label.append(idx2label[v])

INFO:tensorflow:Restoring parameters from ./01_simple_linear_regression/model.ckpt


In [23]:
acc_id = x_test_temp.groupby(by='acc_id').mean().index

data = {'acc_id':acc_id, 'label':label}

In [24]:
result = pd.DataFrame(data)

In [26]:
result.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\result\\01_simple_linear_regression.csv')